<a href="https://colab.research.google.com/github/ancestor9/2025_Spring_Data-Management/blob/main/week_08/Text_Representation_and_Embedding_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텍스트 표현 기법과 임베딩
# **Data Representation**

## <font color='orange'>**4. Text data**
- **아래 그림을 이해하여야 한다.**
<img src='http://jalammar.github.io/images/numpy/numpy-nlp-embeddings.png'>
<img src='http://jalammar.github.io/images/numpy/numpy-nlp-bert-shape.png'>


- **이 그림도 이해하여야 한다.**
<img src ='https://camo.githubusercontent.com/7fd0081ba6f27b73017f5307d663c8bf4e83bbd98e0eb148a34ffdd48be642f5/68747470733a2f2f73332e616d617a6f6e6177732e636f6d2f626f6f6b2e6b657261732e696f2f696d672f6368362f776f72645f656d62656464696e67732e706e67' width =400 height=400>

#### **2.6 사전학습 임베딩**
- Gensim / HuggingFace Transformers 사용 가능
- 예: word2vec-google-news-300
#### **2.6.1 [버트(Bidirectional Encoder Representations from Transformers, BERT)](https://wikidocs.net/115055)**

In [1]:
import pandas as pd
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased") # Bert-base의 사전학습된 토크나이저

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [2]:
result = tokenizer.tokenize('Here is the sentence I want embeddings for.')
print(result)

['here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.']


In [ ]:
print(tokenizer.vocab['here'])

2182


In [4]:
print(tokenizer.vocab['embeddings'])

KeyError: 'embeddings'

In [3]:
print(tokenizer.vocab['##bed'])

8270


### BERT 기반 WordPiece Tokenizer는 다음과 같은 특성

- 토큰을 단어 단위로 자르지 않고 서브워드(subword) 단위로 자릅니다.
- 어휘집(tokenizer.vocab)에는 BERT 학습 시 사용된 서브워드만 포함됩니다.
- 새로운 단어는 ## 접두어가 붙은 서브워드 조각으로 분할됩니다

In [5]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
sentence = "Here is the sentence I want embeddings for."
tokens_wp = tokenizer.tokenize(sentence)
print("WordPiece 토큰:", tokens_wp)

# 각 토큰이 vocab에 있는지 확인하고 인덱스 출력
for token in tokens_wp:
    if token in tokenizer.vocab:
        print(f"{token} -> {tokenizer.vocab[token]}")
    else:
        print(f"{token} is not in vocab")


WordPiece 토큰: ['here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.']
here -> 2182
is -> 2003
the -> 1996
sentence -> 6251
i -> 1045
want -> 2215
em -> 7861
##bed -> 8270
##ding -> 4667
##s -> 2015
for -> 2005
. -> 1012


### 어휘집 크기를 작게 유지하면서도 새로운 단어를 처리할 수 있는 유연성 확보

#### **Tokenize By Sub-Word**
<img src ='https://testerstories.com/files/ai_learn/tokenizing-strategies.jpg'>

<img src ='https://testerstories.com/files/ai_learn/tokenize-middle-ground.jpg'>


**🧠 왜 이렇게 하나?**
- 단어 수를 제한(예: 30,522개)하면서도 유연하게 신조어, 희귀어 처리 가능
- 새로운 단어도 기존 subword 조합으로 처리 가능 → openaiGPTstyle도 잘 분해됨

In [6]:
# BERT의 단어 집합을 vocabulary.txt에 저장
with open('vocabulary.txt', 'w') as f:
  for token in tokenizer.vocab.keys():
    f.write(token + '\n')


In [7]:
df = pd.read_fwf('vocabulary.txt', header=None)
df

,0
0,[PAD]
1,[unused0]
2,[unused1]
3,[unused2]
4,[unused3]
...,...
30517,##．
30518,##／
30519,##：
30520,##？


## **BERT**

In [9]:
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==

In [10]:
# 0. 라이브러리 불러오기
import os
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

# 1. wandb 비활성화
os.environ["WANDB_DISABLED"] = "true"


In [11]:
# 2. IMDb 데이터 로드 및 축소
raw_dataset = load_dataset("imdb")
dataset = raw_dataset["train"].shuffle(seed=42).select(range(100))  # train 데이터 중 100개 사용


# 3. 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# 4. 토큰화 함수 정의
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=256)

tokenized_dataset = dataset.map(tokenize)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [12]:
# 5. Train / Eval 분리
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# 6. 모델 준비
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:

# 7. Trainer 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    evaluation_strategy="epoch",
    logging_strategy="no",
    report_to="none"  # wandb 완전 끄기
)

# 8. Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:

# 9. 학습 실행
trainer.train()

# 10. 예측 테스트
test_text = "The movie was absolutely fantastic!"
inputs = tokenizer(test_text, return_tensors="pt", truncation=True, padding=True, max_length=256)
outputs = model(**inputs)
pred = torch.argmax(outputs.logits).item()
label = "긍정" if pred == 1 else "부정"
print(f"'{test_text}' → {label}")



Epoch,Training Loss,Validation Loss
1,No log,0.657316
2,No log,0.641144


'The movie was absolutely fantastic!' → 긍정


## **HuggingFace**

In [16]:
from transformers import pipeline

# 1. 감성 분석 파이프라인 생성
# distilbert-base-uncased-finetuned-sst-2-english --> default LLM
classifier = pipeline("sentiment-analysis")

# 2. 예측할 문장
text = "The professor explains very clearly and the class is enjoyable."

# 3. 예측
result = classifier(text)
result


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9998881816864014}]

In [17]:
from transformers import pipeline

# 1. 다국어 감성 분석 모델 로드
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# 2. 한국어 문장
texts = [
    "이 강의는 정말 유익하고 재미있었어요.",
    "교수님의 설명이 이해하기 어려웠습니다.",
    "수업이 별로였어요.",
    "내용이 알차고 좋았습니다.",
    "정말 시간 낭비였어요.",
    "이 수업 정말 미치도록 듣기 싫어, 다시는 듣고 싶지 않아"
]

# 3. 예측
for text in texts:
    result = classifier(text)[0]
    print(f"'{text}' → {result['label']} (score: {result['score']:.2f})")


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


'이 강의는 정말 유익하고 재미있었어요.' → 1 star (score: 0.40)
'교수님의 설명이 이해하기 어려웠습니다.' → 3 stars (score: 0.33)
'수업이 별로였어요.' → 2 stars (score: 0.27)
'내용이 알차고 좋았습니다.' → 4 stars (score: 0.37)
'정말 시간 낭비였어요.' → 3 stars (score: 0.25)
'이 수업 정말 미치도록 듣기 싫어, 다시는 듣고 싶지 않아' → 2 stars (score: 0.43)


In [19]:
! pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.6 MB/s eta 0:00:00


In [20]:
# prompt: gradio로 챗봇화면을 만들어줘 classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")로 긍정과 부정을 보내주는 1star별 별 한 개로 그림으로 결과를 보야줘

import gradio as gr
from transformers import pipeline

# 감성 분석 파이프라인 로드
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

def predict_sentiment(text):
    result = classifier(text)[0]
    label = result['label']
    score = result['score']

    # 별점으로 변환 (5점 만점)
    star_rating = int(round(score * 5))
    star_image = "⭐" * star_rating + "☆" * (5 - star_rating)

    return f"{label} ({star_image})"

iface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=2, placeholder="Enter text here..."),
    outputs="text",
    title="Sentiment Analysis with Star Rating",
    description="Enter some text, and the model will classify its sentiment and give a star rating!"
)

iface.launch()


Device set to use cpu


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1b3112fb9597598113.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
